# Overview of Data Science Pipeline Within This Notebook

## 1. Acquire
The get_mvp_home_data() function within the Acquire module:
Connects to the SQL database with env.py login credentials
Checks if the data is already saved to csv file (has the function been run in the current working directory?)
If no .csv file
Connects to the Zillow Database
Reads a sql query that selects the 4 features of single unit properties during May and June
Saves to a csv, and returns the df
If .csv file
Returns df from the csv
After acquiring the data, we can do univariate exploration:
Describe the data
Plot single variable histograms

## 2. Prepare
Within the Prepare module, the data is prepped by:
Renaming columns
Dropping uneeded id column
Removing outliers using z-score to determine
Splitting into train, validate, test
Scaling the data

## 3. Explore
Once data is split, we can perform bivariate analysis on train
Determine what features influence tax value more
Hypothesis testing
Plotting correlation
Plotting bivariate analysis

## 4. Modeling
Create a baseline prediction of tax value
Create models
Fit models on train
Use LinearRegression, LassoLars, Polynomial
Ensure models not overfit, testing top 3 models on validate
Simulate best model being run on future unseen data, test on test df

## 5. Conclusions
Conclude which model preforms best, which features have the highest influence on tax value, and what to do with this information

# Imports 

In [2]:
# Set up the notebook imports
# Import modules
import acquire
import prepare

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# Modeling Imports
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, LassoLars
from sklearn.preprocessing import PolynomialFeatures

# Modeling Metrics
from sklearn.metrics import mean_squared_error, explained_variance_score
from scipy import stats

# Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

print('Imports Completed.')

Imports Completed.
